<a href="https://colab.research.google.com/github/jennif8r/LllForIFPR/blob/main/RAG_NAIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Configuração do ambiente



In [26]:
!pip install langchain langchain_community langchain_openai streamlit openai unstructured tiktoken faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.1 MB/s eta 0:00:00


#apply.py

In [52]:
import os
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import OpenAI

# Configurar a API Key e modelo
os.environ['OPENAI_API_KEY'] = 'Senha da api'
llm_openai = OpenAI(temperature=0.7, max_tokens=300) # usando gpt-3.5-turbo-instruct

# URL fixa
urls = ["https://ifpr.edu.br/colombo/"]
file_name = 'all_url_data_vectors'

user_query = "qual é o Horário das Aulas"

# Carregando dados da URL em forma de texto
url_loader = UnstructuredURLLoader(urls=urls)
url_data = url_loader.load()

# Dividindo os dados carregados em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '.', ' '],
    chunk_size=1000,
)
print("Trabalho em Andamento: Dividindo Texto")
chunked_url_data = text_splitter.split_documents(url_data)

# Criar embeddings usando OpenAI
embedding_creator = OpenAIEmbeddings()

print("Trabalho em Andamento: Criando Embeddings")
data_vectors = FAISS.from_documents(chunked_url_data, embedding_creator)
# Salvar embeddings
data_vectors.save_local(file_name)

if os.path.exists(file_name): # Verificar se o arquivo foi salvo
    print("Trabalho em Andamento: Carregando Resultados")
    # Carregar os vetores de dados
    data_vectors_loaded = FAISS.load_local(file_name, embedding_creator, allow_dangerous_deserialization=True)
    # Consultar o LLM
    main_chain = RetrievalQAWithSourcesChain.from_llm(llm=llm_openai, retriever=data_vectors_loaded.as_retriever())
    llm_result = main_chain({'question': user_query})
    print("Tarefa Concluída: Exibindo Resultados")
    # Exibir a resposta do LLM
    print('Resposta:')
    print(llm_result['answer'])
    # Obtendo a(s) fonte(s) da resposta do LLM
    answer_sources = llm_result.get('sources','') # Verificar se há fontes
    if answer_sources:
        answer_sources_list = answer_sources.split('\n')
        print('Para mais informação acesse o LINK:')
        for source in answer_sources_list:
            print(source)


Trabalho em Andamento: Dividindo Texto
Trabalho em Andamento: Criando Embeddings
Trabalho em Andamento: Carregando Resultados
Tarefa Concluída: Exibindo Resultados
Resposta:
 O horário das aulas no IFPR pode variar entre 7h30 e 22h, mas é importante consultar o site do campus ou o calendário acadêmico para obter informações específicas. No entanto, devido à pandemia, o calendário acadêmico está suspenso desde 01 de maio de 2020. 

Para mais informação acesse o LINK:
https://ifpr.edu.br/colombo/
